maximize: sales = p1 * 1200 + p2 * 2500 + p3 * 1400 
mimize :  - sales = - p1 * 1200 - p2 * 2500 - p3 * 1400 
subject to: 
        1. demand(bnd): p1 <= 35,
                                  p2 <= 25,
                                            p3 <= 30
        2. capacity: 
                   цех1: p1 +    p2 +     p3 <= 40
                   цех2: 0  +    0  +     p3 <= 20 
                   цех3: 0       p2 * 8 + p3 * 2 <= 80
                   
        3. supply: p1 * 0,8  +   p2 * 0,6 + p3 * 0,7 <= 50       

In [13]:
# SciPy
from scipy.optimize import linprog
import numpy as np

In [2]:
obj = [- 1200, - 2500, - 1400]

In [3]:
bnd = [(0, 35), (0, 25), (0, 30)]

In [4]:
lhs_ineq = [[1, 1, 1], [0, 0, 1], [0, 8, 2], [0.8, 0.6, 0.7]]

In [5]:
rhs_ineq = [40, 20, 80, 50]

In [6]:
opt = linprog(c=obj, A_ub = lhs_ineq, b_ub= rhs_ineq, bounds=bnd)

In [7]:
opt

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -61000.0
              x: [ 3.000e+01  1.000e+01  0.000e+00]
            nit: 3
          lower:  residual: [ 3.000e+01  1.000e+01  0.000e+00]
                 marginals: [ 0.000e+00  0.000e+00  1.250e+02]
          upper:  residual: [ 5.000e+00  1.500e+01  3.000e+01]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 0.000e+00  2.000e+01  0.000e+00  2.000e+01]
                 marginals: [-1.200e+03 -0.000e+00 -1.625e+02 -0.000e+00]
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip_gap: 0.0

In [16]:
opt.x

array([30., 10.,  0.])

In [22]:
# check solution:
check = np.array(lhs_ineq).dot(opt.x)
check

array([40.,  0., 80., 30.])

In [77]:
print(f'продажи всего:  {-np.array(obj).dot(opt.x):,}')
print(f'производим П1: {opt.x[0]} при спросе {bnd[0][1]} на сумму {-obj[0]*opt.x[0]};\n\
производим П2: {opt.x[1]} при спросе {bnd[1][1]}  на сумму {-obj[1]*opt.x[1]};\n\
производим П3: {opt.x[2]} при спросе {bnd[2][1]}  на сумму {-obj[2]*opt.x[2]}.\n')

print(f'Ц1 производит {check[0]} при ограничении {rhs_ineq[0]};\n\
Ц2 производит {check[1]} при ограничении {rhs_ineq[1]};\n\
Ц3 производит {check[2]} при ограничении {rhs_ineq[2]};\n\
Всего производится {check[3]} при ограничении {rhs_ineq[3]}.')

продажи всего:  61,000.0
производим П1 30.0 при спросе 35 на сумму 36000.0;
производим П2 10.0 при спросе 25  на сумму 25000.0;
производим П3 0.0 при спросе 30  на сумму 0.0.

Ц1 производит 40.0 при ограничении 40;
Ц2 производит 0.0 при ограничении 20;
Ц3 производит 80.0 при ограничении 80;
Всего производится 30.0 при ограничении 50.


In [12]:
# pulp
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable

In [50]:
model = LpProblem (name = 'factory', sense = LpMaximize)

In [51]:
# demand
p1 = LpVariable(name = 'p1', lowBound = 0, upBound = 35, cat='Integer')
p2 = LpVariable(name = 'p2', lowBound = 0, upBound = 25, cat='Integer')
p3 = LpVariable(name = 'p3', lowBound = 0, upBound = 30, cat='Integer')

In [59]:
# sales
obj_func = (p1 * 1200 + p2 * 2500 + p3 * 1400)
model += obj_func

In [57]:
# capacity:
model += (p1 +    p2 +     p3 <= 40) # цех1
model += (p3 <= 20) #цех2
model += (p2 * 8 + p3 * 2 <= 80) # цех3
# supply:
model += (p1 * 0.8  +   p2 * 0.6 + p3 * 0.7 <= 50)

In [60]:
model

factory:
MAXIMIZE
1200*p1 + 2500*p2 + 1400*p3 + 0
SUBJECT TO
model_supply: 0.8 p1 + 0.6 p2 + 0.7 p3 <= 50

first: p1 + p2 + p3 <= 40

_C1: p1 + p2 + p3 <= 40

_C2: 0.8 p1 + 0.6 p2 + 0.7 p3 <= 50

_C3: p1 + p2 + p3 <= 40

_C4: p3 <= 20

_C5: 8 p2 + 2 p3 <= 80

_C6: 0.8 p1 + 0.6 p2 + 0.7 p3 <= 50

VARIABLES
0 <= p1 <= 35 Integer
0 <= p2 <= 25 Integer
0 <= p3 <= 30 Integer

In [62]:
status = model.solve()

In [63]:
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")

for var in model.variables():
    print(f"{var.name}: {var.value()}")

for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.value()}")

status: 1, Optimal
objective: 61000.0
p1: 30.0
p2: 10.0
p3: 0.0
model_supply: -20.0
first: 0.0
_C1: 0.0
_C2: -20.0
_C3: 0.0
_C4: -20.0
_C5: 0.0
_C6: -20.0
